## Нейронные сети: зависимость ошибки и обучающей способности от числа нейронов

In [1]:
# Выполним инициализацию основных используемых модулей
%matplotlib inline

import random
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
import numpy as np
import sklearn
from tqdm.notebook import tqdm
import tensorflow as tf
import pandas as pd
tf.config.run_functions_eagerly(True)
%load_ext blackcellmagic

In [2]:
from loguru import logger
logger.add('error.message.txt', format="{time} - {level} - {message}")

1

In [15]:
from tensorflow.keras import regularizers
import snoop

Выполним загрузку данных

In [69]:
'''with open('winequality-red.csv') as f:
    f.readline()  # пропуск заголовочной строки
    data = np.loadtxt(f, delimiter=';')
df = pd.DataFrame(data)
df.to_csv('winequality-red1.csv', header=True, index=False)
df = pd.read_csv('winequality-red1.csv')
bins = [0, 5, 7, np.inf]
labels_list = [0, 1, 2]
df['11'] = pd.cut(df['11'], bins=bins, labels=labels_list)
df.to_csv('winequality-red1.csv', header=True, index=False)'''

In [3]:
df = pd.read_csv('winequality-red1.csv')

In [16]:
HIDDEN_NEURONS_NUM = [50, 100, 200, 500, 700, 1000]

In [ ]:
'''batch_size = 1
train_dataset = tf.data.experimental.make_csv_dataset(
    "winequality-red1.csv",
    batch_size,
    select_columns=[str(i) for i in range(12)],
    label_name="11",
    num_epochs=1,
    shuffle=True,
)
features, labels = next(iter(train_dataset))
column_names = [str(i) for i in range(12)]
features = column_names[:-1]
labels = column_names[-1]


def pack_features_vector(features, labels):
    features = tf.stack(list(features.values()), axis=1)
    return features, labels


train_dataset = train_dataset.map(pack_features_vector)
features, labels = next(iter(train_dataset))'''

In [26]:
tf.data.experimental.enable_debug_mode()

In [27]:
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(df.drop('11', axis=1))

In [13]:
column_names = np.array(df.columns)

In [19]:
from collections import defaultdict

In [100]:
%%time
@logger.catch
@tf.function(jit_compile=True)
def experiment():
    results = defaultdict(float)
    es = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", mode="max", verbose=1, patience=3)
    for i in HIDDEN_NEURONS_NUM:
        tf.random.set_seed(0)
        model = get_basic_model(i)
        metrics = model.fit(
            x_train,
            y_train,
            batch_size=1,
            epochs=100,
            use_multiprocessing=True,
            validation_data=(x_test, y_test),
            callbacks=[es],
            verbose=0,
        )
        results[i] = metrics.history
        model.save(f"/Users/glebsokolov/MainDir/C2W5.models/model{i}")
    return results


exp_run = experiment()

Epoch 00007: early stopping


INFO:tensorflow:Assets written to: /Users/glebsokolov/MainDir/C2W5.models/model50/assets


INFO:tensorflow:Assets written to: /Users/glebsokolov/MainDir/C2W5.models/model50/assets


Epoch 00007: early stopping


INFO:tensorflow:Assets written to: /Users/glebsokolov/MainDir/C2W5.models/model100/assets


...

INFO:tensorflow:Assets written to: /Users/glebsokolov/MainDir/C2W5.models/model700/assets


Epoch 00007: early stopping


INFO:tensorflow:Assets written to: /Users/glebsokolov/MainDir/C2W5.models/model1000/assets


INFO:tensorflow:Assets written to: /Users/glebsokolov/MainDir/C2W5.models/model1000/assets


CPU times: user 9min 23s, sys: 7.59 s, total: 9min 31s
Wall time: 9min 39s


In [31]:
def write_answer(optimal_neurons_num):
    with open("nnets_answer1.txt", "w") as fout:
        fout.write(str(optimal_neurons_num))

In [103]:
for i in HIDDEN_NEURONS_NUM:
    print(exp_run[i]['val_accuracy'][-1])

0.736952006816864
0.7390396595001221
0.7327766418457031
0.7348642945289612
0.7348642945289612
0.7348642945289612


In [86]:
write_answer(1000)

In [81]:
@logger.catch
@tf.function(jit_compile=True)
def get_basic_model(N):
    model = tf.keras.Sequential(
        [
            normalizer,
            tf.keras.layers.Dense(N, kernel_regularizer=regularizers.l2(0.001)),
            tf.keras.layers.Dense(3),
        ]
    )
    model.compile(
        optimizer="sgd",
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=["accuracy"],
    )
    return model

In [47]:
x_train = tf.convert_to_tensor(df.drop("11", axis=1).loc[: int(0.7 * len(df))])
y_train = tf.convert_to_tensor(df["11"].loc[: int(0.7 * len(df))])
x_test = tf.convert_to_tensor(df.drop("11", axis=1).loc[int(0.7 * len(df)) + 1 :])
y_test = tf.convert_to_tensor(df["11"].loc[int(0.7 * len(df)) + 1 :])

In [ ]:
tf.one_hot(y, depth=3)

In [18]:
model.fit(train, y, epochs=20, batch_size=1, use_multiprocessing=True)

Epoch 1/20
  11/1599 [..............................] - ETA: 17s - loss: 1.0305 - accuracy: 0.4545

/Users/glebsokolov/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/data/ops/dataset_ops.py:4211: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


1599/1599 [==============================] - 19s 12ms/step - loss: 0.7035 - accuracy: 0.7173
Epoch 2/20
1599/1599 [==============================] - 18s 11ms/step - loss: 0.6514 - accuracy: 0.7305
Epoch 3/20
1599/1599 [==============================] - 17s 11ms/step - loss: 0.6370 - accuracy: 0.7280
Epoch 4/20
1599/1599 [==============================] - 19s 12ms/step - loss: 0.6281 - accuracy: 0.7342
Epoch 5/20
1599/1599 [==============================] - 24s 15ms/step - loss: 0.6228 - accuracy: 0.7317
Epoch 6/20
1599/1599 [==============================] - 18s 11ms/step - loss: 0.6102 - accuracy: 0.7405
Epoch 7/20
1599/1599 [==============================] - 18s 11ms/step - loss: 0.6074 - accuracy: 0.7336
Epoch 8/20
1599/1599 [==============================] - 19s 12ms/step - loss: 0.6006 - accuracy: 0.7348
Epoch 9/20
1599/1599 [==============================] - 20s 12ms/step - loss: 0.5994 - accuracy: 0.7411
Epoch 10/20
1599/1599 [==============================] - 19s 12ms/step - lo

In [71]:
training_df: pd.DataFrame = df
features = column_names[:-1]
train_dataset = (
    tf.data.Dataset.from_tensors(
        (
            x_train,
            y_train
        )
    )
)

In [70]:
features, labels = x_train, y_train

In [72]:
@logger.catch
def prepare():
    global model
    model = tf.keras.Sequential(
        [
            tf.keras.layers.Dense(
                50,
                input_shape=(11,),
                kernel_regularizer=regularizers.l2(0.001),
            ),
            tf.keras.layers.Dense(3),
        ]
    )
    global loss
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

    def grad(model, inputs, targets):
        with tf.GradientTape() as tape:
            logits = model(inputs, training=True)
            loss_value = loss(targets, logits)
        return loss_value, tape.gradient(loss_value, model.trainable_variables)
    global optimizer
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
    loss_value, grads = grad(model, features, labels)

    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return model

In [73]:
model = prepare()

2021-10-11 22:19:33.506 | ERROR    | __main__:<module>:1 - An error has been caught in function '<module>', process 'MainProcess' (61371), thread 'MainThread' (4445056512):
Traceback (most recent call last):

  File "/Users/glebsokolov/opt/anaconda3/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
           │         │     └ {'__name__': '__main__', '__doc__': 'Entry point for launching an IPython kernel.\n\nThis is separate from the ipykernel pack...
           │         └ <code object <module> at 0x7ff47c3487c0, file "/Users/glebsokolov/opt/anaconda3/lib/python3.8/site-packages/ipykernel_launche...
           └ <function _run_code at 0x7ff47c3a0ca0>
  File "/Users/glebsokolov/opt/anaconda3/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
         │     └ {'__name__': '__main__', '__doc__': 'Entry point for launching an IPython kernel.\n\nThis is separate from the ipykernel pack...
         └ <code obje

In [698]:
@logger.catch
def train_steps():
    ## Note: Rerunning this cell uses the same model variables

    # Keep results for plotting
    train_loss_results = []
    train_accuracy_results = []

    num_epochs = 100

    for epoch in tqdm(range(num_epochs)):
        epoch_loss_avg = tf.keras.metrics.Mean()
        epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

        for x, y in train_dataset:
            # Optimize the model
            loss_value, grads = grad(model, x, y)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))

            # Track progress
            epoch_loss_avg.update_state(loss_value)  # Add current batch loss
            # Compare predicted label to actual label
            # training=True is needed only if there are layers with different
            # behavior during training versus inference (e.g. Dropout).
            epoch_accuracy.update_state(y, model(x, training=True))

        # End epoch
    train_loss_results.append(epoch_loss_avg.result())
    train_accuracy_results.append(epoch_accuracy.result())

    return (
        print(
            "Epoch {:03d}: Loss: {:.3f}, Accuracy: {:.3%}".format(
                epoch, epoch_loss_avg.result(), epoch_accuracy.result()
            )
        ),
        train_loss_results,
        train_accuracy_results,
    )

In [699]:
train_steps()

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 009: Loss: 0.777, Accuracy: 54.784%


(None,
 [<tf.Tensor: shape=(), dtype=float32, numpy=0.77678907>],
 [<tf.Tensor: shape=(), dtype=float32, numpy=0.5478424>])

In [581]:
model.summary()

Model: "sequential_104"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_287 (Dense)            (None, 50)                600       
_________________________________________________________________
dense_288 (Dense)            (None, 50)                2550      
_________________________________________________________________
dense_289 (Dense)            (None, 3)                 153       
Total params: 3,303
Trainable params: 3,303
Non-trainable params: 0
_________________________________________________________________
